In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import SRW
from collections import Counter

# This toy example is for checking the partial derivatives

In [4]:
edges = [[0,1],[0,2],[1,2],[1,0],[2,0],[2,1]]
features = np.array([[0.9,0.1],[0.1,0.9],[0.5,0.5],[0.9,0.1],[0.1,0.9],[0.5,0.5]])
w0 = np.array([0.7,0.3])
w1 = np.array([0.7,0.300001])
rst_prob = 0.5
P_init = mutation_profile = [[1,0,0],[0,1,0],[0,1,1],[1,1,0],[1,1,1]]
nnodes = 3
group_lables = [0, 0, 1, 1, 1]
lam = 0.001

In [5]:
Q0, Q_grad = SRW.generate_Q_and_gradient(edges, nnodes, features, w0)
Q1, Q_grad1 = SRW.generate_Q_and_gradient(edges, nnodes, features, w1)

In [6]:
Q_grad

array([[[ 0.        ,  0.06602963, -0.06602963],
        [ 0.02944953,  0.        , -0.02944953],
        [-0.03676033,  0.03676033,  0.        ]],

       [[ 0.        , -0.08472869,  0.08472869],
        [-0.03864221,  0.        ,  0.03864221],
        [ 0.04631791, -0.04631791,  0.        ]]])

In [7]:
(Q1 - Q0) / 0.000001 # check

array([[ 0.        , -0.08472867,  0.08472867],
       [-0.0386422 ,  0.        ,  0.0386422 ],
       [ 0.0463179 , -0.0463179 ,  0.        ]])

In [8]:
P0 = SRW.iterative_PPR(Q0, mutation_profile, rst_prob)
P1 = SRW.iterative_PPR(Q1, mutation_profile, rst_prob)

In [9]:
P_grad = SRW.iterative_P_gradient(P0, Q0, Q_grad, rst_prob)
P_grad

array([[[ -1.37842111e-04,   1.85449268e-02,  -1.84070847e-02],
        [  4.38374474e-03,   8.16865825e-03,  -1.25524030e-02],
        [ -9.14792575e-04,   1.10144385e-02,  -1.00996460e-02],
        [  2.12295131e-03,   1.33567925e-02,  -1.54797438e-02],
        [ -6.55809087e-04,   1.35246013e-02,  -1.28687922e-02]],

       [[  4.03436264e-05,  -2.37325578e-02,   2.36922142e-02],
        [ -5.89826629e-03,  -1.04177323e-02,   1.63159986e-02],
        [  9.00786327e-04,  -1.40012357e-02,   1.31004493e-02],
        [ -2.92896133e-03,  -1.70751451e-02,   2.00041064e-02],
        [  6.13972093e-04,  -1.72450097e-02,   1.66310376e-02]]])

In [10]:
(P1 - P0) / 0.000001 # check

array([[  4.00703914e-05,  -2.37318405e-02,   2.36917700e-02],
       [ -5.89760663e-03,  -1.04185502e-02,   1.63161568e-02],
       [  9.00890473e-04,  -1.40015377e-02,   1.31006472e-02],
       [ -2.92876801e-03,  -1.70751954e-02,   2.00039634e-02],
       [  6.13950391e-04,  -1.72449720e-02,   1.66310215e-02]])

In [11]:
group2npairs = SRW.count_numbers_per_group(group_lables)

In [12]:
J0 = SRW.obj_func(P0, group_lables, group2npairs, lam, w0)
J1 = SRW.obj_func(P1, group_lables, group2npairs, lam, w1)

In [13]:
J_grad = SRW.obj_func_gradient(Q0, Q_grad, P0, P_grad, group_lables, group2npairs, lam, w0)
J_grad

array([-0.00884211,  0.01386413])

In [14]:
(J1 - J0) / 0.000001  # check

0.013862237024930835

In [20]:
J0 = SRW.obj_func(P0, group_lables, group2npairs, lam, w0, 'absolute', 'L1', True)
J1 = SRW.obj_func(P1, group_lables, group2npairs, lam, w1, 'absolute', 'L1', True)

In [23]:
J_grad = SRW.obj_func_gradient(Q0, Q_grad, P0, P_grad, group_lables, group2npairs, lam, w0, 'absolute', 'L1', True)
J_grad

array([-0.00049987,  0.00312944])

In [24]:
(J1 - J0) / 0.000001  # check

0.0031285922741375316

# This toy example is for checking the gradient descent functions

In [150]:
edges = [[0,1],[0,2],[0,3],[0,4],[0,5],[1,2],[1,3],[1,4],[1,5],[2,3],[2,4],[2,5],[3,4],[3,5],[4,5],
         [1,0],[2,0],[3,0],[4,0],[5,0],[2,1],[3,1],[4,1],[5,1],[3,2],[4,2],[5,2],[4,3],[5,3],[5,4],
         [0,0],[1,1],[2,2],[3,3],[4,4],[5,5]]
features = np.array([[.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                     [.9,.4,0.,1.],[.9,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.5,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.1,.4,0.,1.],[.1,.5,0.,1.],[.1,.6,0.,1.],[.9,.4,0.,1.],[.9,.6,0.,1.],[.9,.5,0.,1.],
                     [.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.],[.0,.0,1.,1.]])
rst_prob = 0.15
P_init = mutation_profile = [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],
                             [0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]
nnodes = 6
group_lables = [0, 0, 0, 1, 1, 1]
lam = 1e-2
w_init_sd = 0.01
learning_rate = 0.001

In [151]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_lables, lam, 
                     w_init_sd, 'absolute', 'L1', True, learning_rate=0.1, update_w_func='Nadam')

0 iteration: J is  0.299881052804
1 iteration: J is  0.293656204843
2 iteration: J is  0.289159352616
3 iteration: J is  0.285466755373
4 iteration: J is  0.28201002852
5 iteration: J is  0.279692065652
6 iteration: J is  0.276137568838
7 iteration: J is  0.272836207268
8 iteration: J is  0.269089322353
9 iteration: J is  0.26482206925
10 iteration: J is  0.260163983951
11 iteration: J is  0.255206197768
12 iteration: J is  0.249215018925
13 iteration: J is  0.243730995766
14 iteration: J is  0.23704232717
15 iteration: J is  0.229813599528
16 iteration: J is  0.222182497247
17 iteration: J is  0.214127851738
18 iteration: J is  0.205538634644
19 iteration: J is  0.19597948865
20 iteration: J is  0.185728073304
21 iteration: J is  0.174693130982
22 iteration: J is  0.162773882821
23 iteration: J is  0.149863487017
24 iteration: J is  0.135857733286
25 iteration: J is  0.120631356603
26 iteration: J is  0.104064572402
27 iteration: J is  0.0860438972775
28 iteration: J is  0.06644655303

233 iteration: J is  -1.36266284953
234 iteration: J is  -1.36290524062
235 iteration: J is  -1.36268050589
236 iteration: J is  -1.36268417157


In [152]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_lables, lam, 
                     w_init_sd, 'absolute', 'L1', True, learning_rate=0.1, update_w_func='Nesterov')

0 iteration: J is  0.30030355201
1 iteration: J is  0.300147627381
2 iteration: J is  0.300066387116
3 iteration: J is  0.299963237279
4 iteration: J is  0.299828166752
5 iteration: J is  0.299544150419
6 iteration: J is  0.298540326084
7 iteration: J is  0.297292907105
8 iteration: J is  0.295814899312
9 iteration: J is  0.294156929686
10 iteration: J is  0.292374027167
11 iteration: J is  0.290429876706
12 iteration: J is  0.28839789843
13 iteration: J is  0.286310638718
14 iteration: J is  0.284181481162
15 iteration: J is  0.281991213207
16 iteration: J is  0.279798037029
17 iteration: J is  0.277613642205
18 iteration: J is  0.275408764372
19 iteration: J is  0.273275126429
20 iteration: J is  0.271104960948
21 iteration: J is  0.269016230096
22 iteration: J is  0.266918540725
23 iteration: J is  0.264840166047
24 iteration: J is  0.262838188424
25 iteration: J is  0.260797938514
26 iteration: J is  0.258819628105
27 iteration: J is  0.256849621687
28 iteration: J is  0.2548814262

238 iteration: J is  -1.35266212435
239 iteration: J is  -1.35272956644
240 iteration: J is  -1.35277249603
241 iteration: J is  -1.35284063124
242 iteration: J is  -1.35288835676
243 iteration: J is  -1.35295441835
244 iteration: J is  -1.35300145336
245 iteration: J is  -1.35306891548
246 iteration: J is  -1.35310940778
247 iteration: J is  -1.35317430966
248 iteration: J is  -1.3532276695
249 iteration: J is  -1.3532914599
250 iteration: J is  -1.35333155538
251 iteration: J is  -1.35339877625
252 iteration: J is  -1.35343938618
253 iteration: J is  -1.35350220259
254 iteration: J is  -1.35355636373
255 iteration: J is  -1.35361550867
256 iteration: J is  -1.35365918128
257 iteration: J is  -1.35372579762
258 iteration: J is  -1.35375835963
259 iteration: J is  -1.35381964746
260 iteration: J is  -1.35388139796
261 iteration: J is  -1.35393009371
262 iteration: J is  -1.35398277991
263 iteration: J is  -1.35403850494
264 iteration: J is  -1.35408479172
265 iteration: J is  -1.354145

472 iteration: J is  -1.363612388
473 iteration: J is  -1.36365545395
474 iteration: J is  -1.36369359788
475 iteration: J is  -1.3637461332
476 iteration: J is  -1.3637759673
477 iteration: J is  -1.36383234203
478 iteration: J is  -1.36385699633
479 iteration: J is  -1.36390823282
480 iteration: J is  -1.36395797511
481 iteration: J is  -1.36399918318
482 iteration: J is  -1.36403981704
483 iteration: J is  -1.36408893764
484 iteration: J is  -1.36412265674
485 iteration: J is  -1.36417771581
486 iteration: J is  -1.36420629441
487 iteration: J is  -1.36426318703
488 iteration: J is  -1.36428630977
489 iteration: J is  -1.36433691917
490 iteration: J is  -1.36439001385
491 iteration: J is  -1.36442676694
492 iteration: J is  -1.36447240337
493 iteration: J is  -1.36451559091
494 iteration: J is  -1.3645556355
495 iteration: J is  -1.36460357812
496 iteration: J is  -1.3646395401
497 iteration: J is  -1.36469088009
498 iteration: J is  -1.36472397939
499 iteration: J is  -1.3647776195

In [153]:
w = SRW.train_SRW_GD(edges, features, nnodes, P_init, rst_prob, group_lables, lam, 
                     w_init_sd, 'absolute', 'L1', True, learning_rate=0.1, update_w_func='Adam')

0 iteration: J is  0.300630885661
1 iteration: J is  0.296688303232
2 iteration: J is  0.289449660608
3 iteration: J is  0.282475869048
4 iteration: J is  0.27706555346
5 iteration: J is  0.271346188684
6 iteration: J is  0.265044831714
7 iteration: J is  0.2580418575
8 iteration: J is  0.250221463516
9 iteration: J is  0.243029754272
10 iteration: J is  0.235276453512
11 iteration: J is  0.226808954208
12 iteration: J is  0.217595774775
13 iteration: J is  0.207575129004
14 iteration: J is  0.196646483458
15 iteration: J is  0.184684289815
16 iteration: J is  0.17153071247
17 iteration: J is  0.157013148131
18 iteration: J is  0.140936035366
19 iteration: J is  0.123107516839
20 iteration: J is  0.103331678991
21 iteration: J is  0.0814293297386
22 iteration: J is  0.05722174432
23 iteration: J is  0.0305544939316
24 iteration: J is  0.00129412458541
25 iteration: J is  -0.0306628296633
26 iteration: J is  -0.065376993158
27 iteration: J is  -0.102882707485
28 iteration: J is  -0.1431

In [154]:
w

array([ 9.52117741, -0.03183526, -0.02135635, -8.81179904])

In [155]:
w, J, d = SRW.train_SRW_BFGS(edges, features, nnodes, P_init, rst_prob, group_lables, lam, w_init_sd=0.01, 
                             loss='absolute', norm_type='L1', maximize_diff=True)

In [156]:
w, J, d

(array([  9.74307199e+00,  -1.30783448e-07,   3.93452998e-09,
         -9.08601073e+00]),
 -1.3665899981072986,
 {'funcalls': 100,
  'grad': array([  7.43052765e-08,  -7.54324526e-03,   1.00474052e-02,
           6.45987717e-07]),
  'nit': 27,
  'task': 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'warnflag': 0})

### Check the transition matrix Q and propagation scores P

In [157]:
Q = SRW.generate_Q(edges, nnodes, features, w)
Q

array([[  1.34200704e-04,   4.99399690e-01,   4.99399682e-01,
          3.55475698e-04,   3.55475693e-04,   3.55475688e-04],
       [  4.99399688e-01,   1.34200704e-04,   4.99399684e-01,
          3.55475696e-04,   3.55475692e-04,   3.55475687e-04],
       [  4.99399684e-01,   4.99399688e-01,   1.34200705e-04,
          3.55475699e-04,   3.55475694e-04,   3.55475690e-04],
       [  3.55475698e-04,   3.55475698e-04,   3.55475698e-04,
          1.34200704e-04,   4.99399690e-01,   4.99399682e-01],
       [  3.55475692e-04,   3.55475692e-04,   3.55475692e-04,
          4.99399688e-01,   1.34200704e-04,   4.99399684e-01],
       [  3.55475690e-04,   3.55475690e-04,   3.55475690e-04,
          4.99399684e-01,   4.99399688e-01,   1.34200705e-04]])

In [158]:
P = SRW.iterative_PPR(Q, P_init, rst_prob)
P

array([[ 0.40154937,  0.29624007,  0.29624007,  0.00199017,  0.00199017,
         0.00199017],
       [ 0.29624007,  0.40154937,  0.29624007,  0.00199017,  0.00199017,
         0.00199017],
       [ 0.29624007,  0.29624007,  0.40154937,  0.00199017,  0.00199017,
         0.00199017],
       [ 0.00199017,  0.00199017,  0.00199017,  0.40154937,  0.29624007,
         0.29624007],
       [ 0.00199017,  0.00199017,  0.00199017,  0.29624007,  0.40154937,
         0.29624007],
       [ 0.00199017,  0.00199017,  0.00199017,  0.29624007,  0.29624007,
         0.40154937]])

In [159]:
group2npairs = SRW.count_numbers_per_group(group_lables)

In [160]:
J = SRW.obj_func(P, group_lables, group2npairs, lam, w, loss='absolute', norm_type='L1', maximize_diff=True)
J

-1.3665899981072986

### Compare with unweighted random walk

In [161]:
Qu = SRW.generate_Q_unweighted(edges, nnodes)
Qu

array([[ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667]])

In [162]:
Pu = SRW.iterative_PPR(Qu, P_init, rst_prob)
Pu

array([[ 0.29166667,  0.14166667,  0.14166667,  0.14166667,  0.14166667,
         0.14166667],
       [ 0.14166667,  0.29166667,  0.14166667,  0.14166667,  0.14166667,
         0.14166667],
       [ 0.14166667,  0.14166667,  0.29166667,  0.14166667,  0.14166667,
         0.14166667],
       [ 0.14166667,  0.14166667,  0.14166667,  0.29166667,  0.14166667,
         0.14166667],
       [ 0.14166667,  0.14166667,  0.14166667,  0.14166667,  0.29166667,
         0.14166667],
       [ 0.14166667,  0.14166667,  0.14166667,  0.14166667,  0.14166667,
         0.29166667]])

In [163]:
Ju = SRW.obj_func(Pu, group_lables, group2npairs, lam, 0, loss='absolute', norm_type='L1', maximize_diff=True)
Ju

0.29999999999999993

# Scratch